In [2]:




import pandas as pd
import collections

Filtering steps:
1. Containing ",", and the string after "," is a state in the US.
2. One of the cities in the list of 754 cities with a 50,000+ population in 2016 (https://www.governing.com/archive/population-density-land-area-cities-map.html, data from U.S. Census Bureau)
3. Counties of the US

# 1 Location data

## 1.1 Construct lowercase city-state dictionary with only populated cities

In [156]:
city_table_raw_small = pd.read_excel('../../Paper data/city list.xlsx') 
city_table_raw_small = city_table_raw_small.replace('hawaiʻi','hawaii')

In [157]:
city_list_lower = []
city_list_lower_state = []
for c in city_table_raw_small.City:
    state = c.rpartition(',')[2].strip()
    city = c.rpartition(',')[0].strip()
    city = city.replace('city','').strip()
    city_list_lower.append(city.lower())
    city_list_lower_state.append(state.lower())

city_dict_small = dict(zip(city_list_lower,city_list_lower_state))

## 1.2 Construct lowercase city-state dictionary with all cities

https://simplemaps.com/data/us-cities

In [158]:
city_table_raw = pd.read_csv('../../Paper data/simplemaps_uscities_basicv1.74/uscities.csv')  
city_table_raw = city_table_raw.replace('hawaiʻi','hawaii')
city_list_lower = []
city_list_lower_state = []
for i,c in city_table_raw.iterrows():
    city = c[0].lower()
    city = city.replace('city','').strip()
    city = city.replace(u'\xf1', u' ')
    city_list_lower.append(city)
    city_list_lower_state.append(c[3].lower())

city_dict = dict(zip(city_list_lower,city_list_lower_state))
dict(list(city_dict.items()))

{'new york': 'new york',
 'los angeles': 'california',
 'chicago': 'illinois',
 'miami': 'missouri',
 'dallas': 'south dakota',
 'philadelphia': 'tennessee',
 'houston': 'arkansas',
 'atlanta': 'nebraska',
 'washington': 'california',
 'boston': 'indiana',
 'phoenix': 'illinois',
 'seattle': 'washington',
 'san francisco': 'california',
 'detroit': 'illinois',
 'san diego': 'texas',
 'minneapolis': 'kansas',
 'tampa': 'kansas',
 'denver': 'missouri',
 'brooklyn': 'illinois',
 'queens': 'new york',
 'riverside': 'wyoming',
 'baltimore': 'ohio',
 'las vegas': 'new mexico',
 'portland': 'iowa',
 'san antonio': 'new mexico',
 'st. louis': 'oklahoma',
 'sacramento': 'new mexico',
 'orlando': 'oklahoma',
 'san jose': 'illinois',
 'cleveland': 'north dakota',
 'pittsburgh': 'pennsylvania',
 'austin': 'nevada',
 'cincinnati': 'iowa',
 'kansas': 'ohio',
 'manhattan': 'montana',
 'indianapolis': 'indiana',
 'columbus': 'illinois',
 'charlotte': 'iowa',
 'virginia beach': 'virginia',
 'bronx': 'n

In [159]:
city_list = list(city_dict.keys())

textfile = open("city_list.txt", "w")
for element in city_list:
    textfile.write(element + ", ")
textfile.close()

## 1.3 Construct the lowercase county-state dictionary

In [160]:
county_table_raw = pd.read_excel('../../Paper data/county list.xlsx', index_col=0) 
county_table_raw = county_table_raw.replace('hawaiʻi','hawaii') 

In [161]:
# Filling missing states
states = []
for i,row in county_table_raw.iterrows():
    if row[0] == row[0]:
        state = row[0]
        state = state.replace(u'\xa0', u' ')
    states.append(state.strip())
county_table_raw['state_filled'] = states

# Lowercase county and state
county_list = list(county_table_raw.index)
county_list_state = list(county_table_raw['state_filled'])
county_list_lower = []
county_list_lower_state = []
for c in county_list:
    if ',' in c:
        c = c.rpartition(',')[0]
    while '[' in c:
        c = c.rpartition('[')[0]
    county_list_lower.append(c.strip().lower())
for c in county_list_state:
    county_list_lower_state.append(c.lower())

county_dict = dict(zip(county_list_lower,county_list_lower_state))
county_dict.pop('new york',None)
dict(list(county_dict.items())[14:19])

{'cleburne': 'arkansas',
 'coffee': 'tennessee',
 'colbert': 'alabama',
 'conecuh': 'alabama',
 'coosa': 'alabama'}

## 1.4 The state abb list

In [162]:
state_table = pd.read_excel('../../Paper data/state list.xlsx', index_col=0)  
state_table = state_table.replace('hawaiʻi','hawaii') 

state_name_list = list(state_table.index)
state_name_list_lower = []
for c in state_name_list:
    state_name_list_lower.append(c.strip().lower())

state_abb_list = list(state_table['state abb'])
state_abb_list_lower = []
for c in state_abb_list:
    state_abb_list_lower.append(c.strip().lower())

state_dict = dict(zip(state_abb_list_lower,state_name_list_lower))

# 2 Read the twitter data

In [3]:
tweets_df_1 = pd.read_csv("../tweets scrapping/tweets_already_scrapped.csv")
tweets_df_2 = pd.read_csv("../tweets scrapping/tweets_df_final_1.csv")
tweets_df = pd.concat([tweets_df_1,tweets_df_2],ignore_index=True)
tweets_df = tweets_df.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1)

In [4]:
len(tweets_df)

1516565

In [10]:
pd.Series(collections.Counter(tweets_df['Keyword'])).to_csv('Distribution of terms.csv')

# 3 Retrive user location information

## 3.1 A dictionary of unique location county/city-state pair

In [163]:
county_dict_unique = dict()
repeated_keys = []
for county,state in county_dict.items():
    if county in county_dict_unique.keys():
        repeated_keys.append(county)
    else:
        county_dict_unique[county] = state

print(len(repeated_keys))

for city,state in city_dict_small.items():
    if city in county_dict_unique.keys():
        repeated_keys.append(city)

print(len(repeated_keys))

repeated_keys_set = set(repeated_keys)

for key in repeated_keys_set:
    county_dict_unique.pop(key,None)

0
165


In [164]:
print(len(county_dict),len(county_dict_unique))

1962 1797


In [165]:
city_dict_unique = dict()
repeated_keys = []

for city,state in city_dict.items():
    if city in city_dict_unique.keys():
        repeated_keys.append(city)
    else:
        city_dict_unique[city] = state

print(len(repeated_keys))

for county,state in county_dict.items():
    if county in city_dict_unique.keys():
        repeated_keys.append(county)

print(len(repeated_keys))

repeated_keys_set = set(repeated_keys)

for key in repeated_keys_set:
    city_dict_unique.pop(key,None)

0
1246


In [166]:
city_dict_unique['new york']

'new york'

In [167]:
print(len(city_dict),len(city_dict_unique))

18837 17591


## 3.2 Construct state-county and state-city dictionary

In [168]:
state_county_dict = dict()
for k,v in county_dict.items():
    if v in state_county_dict.keys():
        state_county_dict[v].append(k)
    else:
        state_county_dict[v] = []
        state_county_dict[v].append(k)

state_county_dict_unique = dict()
for v,k in county_dict_unique.items():
    if k in state_county_dict_unique.keys():
        state_county_dict_unique[k].append(v)
    else:
        state_county_dict_unique[k] = []
        state_county_dict_unique[k].append(v)

state_city_dict = dict()
for v,k in city_dict.items():
    if k in state_city_dict.keys():
        state_city_dict[k].append(v)
    else:
        state_city_dict[k] = []
        state_city_dict[k].append(v)

state_city_dict_unique = dict()
for v,k in city_dict_unique.items():
    if k in state_city_dict_unique.keys():
        state_city_dict_unique[k].append(v)
    else:
        state_city_dict_unique[k] = []
        state_city_dict_unique[k].append(v)

In [173]:
state_county_dict['hawaii'] = state_county_dict['hawaiʻi']

## 3.3 Identify the locations

In [18]:
tweets_sample = tweets_df.sample(n=10)

In [19]:
tweets_sample['User_location']

1055523                         NaN
259560                           19
1510644         The City by The Bay
976931                          NaN
1130982                  Queens, NY
255923     British Columbia, Canada
421422                          NaN
1166488           São Paulo, Brasil
226093                          NaN
1309122                        Hell
Name: User_location, dtype: object

In [174]:
cities = []
counties = []
states = []

for location in tweets_df['User_location']:
    is_valid = False
    if pd.isna(location) is False:
        loc = location.lower().strip()
        if '/' in loc:
            loc = loc.rpartition('/')[0].strip()
        if len(loc) != 0:
            if ',' in loc:
                state = loc.rpartition(',')[2].strip()
                if (state in state_name_list_lower) or (state in state_abb_list_lower):
                    if state in state_name_list_lower:
                        states.append(state)
                    else:
                        states.append(state_dict[state])
                        state = state_dict[state]
                    city_county = loc.rpartition(',')[0].strip()
                    if "county" in city_county:
                        city_county = city_county.rpartition('county')[0].strip()
                    if "city" in city_county:
                        city_county = city_county.rpartition('city')[0].strip()
                    if city_county in state_county_dict[state]:
                        counties.append(city_county)
                        cities.append(None)
                    elif city_county in state_city_dict[state]:
                        cities.append(city_county)
                        counties.append(None)
                    else:
                        cities.append(None)
                        counties.append(None)
                    is_valid = True
                else:
                    state = loc.rpartition(',')[0].strip()
                    if (state in state_name_list_lower) or (state in state_abb_list_lower):
                        if state in state_name_list_lower:
                            states.append(state)
                        else:
                            states.append(state_dict[state])
                        counties.append(None)
                        cities.append(None)
                        is_valid = True
                    
            elif loc in state_abb_list_lower:
                states.append(state_dict[loc])
                cities.append(None)
                counties.append(None)
                is_valid = True
            elif loc in state_name_list_lower:
                states.append(loc)
                cities.append(None)
                counties.append(None)
                is_valid = True
            elif loc in list(county_dict_unique.keys()):
                states.append(county_dict_unique[loc])
                counties.append(loc)
                cities.append(None)
                is_valid = True
            elif loc in list(city_dict_unique.keys()):
                states.append(city_dict_unique[loc])
                cities.append(loc)
                counties.append(None)
                is_valid = True
    if is_valid == False:
        states.append(None)
        cities.append(None)
        counties.append(None)

In [291]:
'''
cities = []
counties = []
states = []

for location in tweets_df['User_location']:
    is_valid = False
    if pd.isna(location) is False:
        loc = location.lower().strip()
        if '/' in loc:
            loc = loc.rpartition('/')[0].strip()
        if len(loc) != 0:
            if ',' in loc:
                state = loc.rpartition(',')[2].strip()
                if (state in state_name_list_lower) or (state in state_abb_list_lower):
                    if state in state_name_list_lower:
                        states.append(state)
                    else:
                        states.append(state_dict[state])
                    city_county = loc.rpartition(',')[0].strip()
                    if "county" in city_county:
                        city_county = city_county.rpartition('county')[0].strip()
                    if "city" in city_county:
                        city_county = city_county.rpartition('city')[0].strip()
                    if city_county in list(county_dict.keys()):
                        counties.append(city_county)
                        cities.append(None)
                    elif city_county in list(city_dict.keys()):
                        cities.append(city_county)
                        counties.append(None)
                    else:
                        cities.append(None)
                        counties.append(None)
                    is_valid = True
                else:
                    state = loc.rpartition(',')[0].strip()
                    if (state in state_name_list_lower) or (state in state_abb_list_lower):
                        if state in state_name_list_lower:
                            states.append(state)
                        else:
                            states.append(state_dict[state])
                        counties.append(None)
                        cities.append(None)
                        is_valid = True
                    
            elif loc in state_abb_list_lower:
                states.append(state_dict[loc])
                cities.append(None)
                counties.append(None)
                is_valid = True
            elif loc in state_name_list_lower:
                states.append(loc)
                cities.append(None)
                counties.append(None)
                is_valid = True
            elif loc in list(county_dict.keys()):
                states.append(county_dict[loc])
                counties.append(loc)
                cities.append(None)
                is_valid = True
            elif loc in list(city_dict.keys()):
                states.append(city_dict[loc])
                cities.append(loc)
                counties.append(None)
                is_valid = True
    if is_valid == False:
        states.append(None)
        cities.append(None)
        counties.append(None)
'''

In [148]:
tweets_sample['User_location']

272894                 pt, eng 
1217081       San Francisco, CA
726824          "New" so "York"
352082                  Glasgow
194506          she/her | 15 | 
798955         Turnersville, NJ
310807                      HTX
446055       𝐞𝐱𝐨 ֊ 𝐧𝐜𝐭 ֊ 𝐤𝐜𝐜 ◞ 
1457623            Turtleshell 
522035              Brooklyn,NY
Name: User_location, dtype: object

In [149]:
print(states)
print(counties)
print(cities)

[None, 'california', None, 'pennsylvania', None, 'new jersey', None, None, None, 'new york']
[None, 'san francisco', None, None, None, None, None, None, None, None]
[None, None, None, 'glasgow', None, 'turnersville', None, None, None, None]


In [150]:
print(len(states),len(counties),len(cities))

10 10 10


In [175]:
has_state = []
has_county = []
has_city = []
for s,county,city in zip(states,counties,cities):
    if s is None:
        has_state.append(False)
    else:
        has_state.append(True)
    if county is None:
        has_county.append(False)
    else:
        has_county.append(True)
    if city is None:
        has_city.append(False)
    else:
        has_city.append(True)

In [176]:
tweets_df['state'] = states
tweets_df['city'] = cities
tweets_df['county'] = counties
tweets_df['has_state'] = has_state
tweets_df['has_county'] = has_county
tweets_df['has_city'] = has_city

In [294]:
print(sum(tweets_df['has_state']),sum(tweets_df['has_county']),sum(tweets_df['has_city']))

315151 90598 131504


# 4 Population data

## 4.1 State population data

In [295]:
state_population_raw = pd.read_excel('../../Paper data/state population.xlsx', index_col=0)  
states_lower = list(state_population_raw['State'].apply(lambda x: x.strip().lower()))
states_population = list(state_population_raw['2020 Census'])
state_population_dict = dict(zip(states_lower,states_population))
state_population_series = pd.DataFrame.from_dict(state_population_dict, orient='index')
state_population_series.rename(columns={0: "population"},inplace=True)

In [60]:
state_population_series[:10]

,population
california,39538223
texas,29145505
florida,21538187
new york,20201249
pennsylvania,13002700
illinois,12801989
ohio,11799448
georgia,10711908
north carolina,10439388
michigan,10077331


## 4.2 County population data

from U.S. Census Bureau https://www.census.gov/data/datasets/time-series/demo/popest/2010s-counties-total.html#par_textimage_739801612

In [66]:
county_population = pd.read_excel('../../Paper data/county population.xlsx')  

In [314]:
counties = []
for county in county_population['CTYNAME']:
    c = county.lower()
    c = c.rpartition('county')[0].strip()
    counties.append(c)
county_population['county'] = counties

In [315]:
county_population

,STNAME,CTYNAME,CENSUS2010POP,POPESTIMATE2019,county
0,Alabama,Alabama,4779736,4903185,
1,Alabama,Autauga County,54571,55869,autauga
2,Alabama,Baldwin County,182265,223234,baldwin
3,Alabama,Barbour County,27457,24686,barbour
4,Alabama,Bibb County,22915,22394,bibb
...,...,...,...,...,...
3188,Wyoming,Sweetwater County,43806,42343,sweetwater
3189,Wyoming,Teton County,21294,23464,teton
3190,Wyoming,Uinta County,21118,20226,uinta
3191,Wyoming,Washakie County,8533,7805,washakie


## 4.3 City population data

Annual Estimates of the Resident Population for Incorporated Places of 50,000 or More, Ranked by July 1, 2019 Population: April 1, 2010 to July 1, 2019, from U.S. Census Bureau. https://www.census.gov/data/tables/time-series/demo/popest/2010s-total-cities-and-towns.html 

In [356]:
city_population = pd.read_excel('../../Paper data/city population.xlsx')

In [357]:
city_list_lower = []
city_list_lower_state = []
for c in city_population['Geographic Area']:
    state = c.rpartition('city,')[2].strip()
    city = c.rpartition('city,')[0].strip()
    city_list_lower.append(city.lower())
    city_list_lower_state.append(state.lower())

city_population['city'] = city_list_lower
city_population['state'] = city_list_lower_state

In [358]:
city_population

,Rank,Geographic Area,2010 Census,2019 Estimate,city,state
0,1,"New York city, New York",8175133.0,8336817,new york,new york
1,2,"Los Angeles city, California",3792621.0,3979576,los angeles,california
2,3,"Chicago city, Illinois",2695598.0,2693976,chicago,illinois
3,4,"Houston city, Texas",2099451.0,2320268,houston,texas
4,5,"Phoenix city, Arizona",1445632.0,1680992,phoenix,arizona
...,...,...,...,...,...,...
783,784,"Lakewood city, Ohio",52131.0,49678,lakewood,ohio
784,785,"Troy city, New York",50129.0,49154,troy,new york
785,786,"Saginaw city, Michigan",51508.0,48115,saginaw,michigan
786,787,"Niagara Falls city, New York",50193.0,47720,niagara falls,new york


# 5 Get the "population" from the "the twitter" dataset

In [177]:
the_tweets = pd.read_csv('../tweets scrapping/tweets_df_the.csv')

## 5.1 Retrive the location!

In [178]:
cities = []
counties = []
states = []

for location in the_tweets['User_location']:
    is_valid = False
    if pd.isna(location) is False:
        loc = location.lower().strip()
        if '/' in loc:
            loc = loc.rpartition('/')[0].strip()
        if len(loc) != 0:
            if ',' in loc:
                state = loc.rpartition(',')[2].strip()
                if (state in state_name_list_lower) or (state in state_abb_list_lower):
                    if state in state_name_list_lower:
                        states.append(state)
                    else:
                        states.append(state_dict[state])
                        state = state_dict[state]
                    city_county = loc.rpartition(',')[0].strip()
                    if "county" in city_county:
                        city_county = city_county.rpartition('county')[0].strip()
                    if "city" in city_county:
                        city_county = city_county.rpartition('city')[0].strip()
                    if city_county in state_county_dict[state]:
                        counties.append(city_county)
                        cities.append(None)
                    elif city_county in state_city_dict[state]:
                        cities.append(city_county)
                        counties.append(None)
                    else:
                        cities.append(None)
                        counties.append(None)
                    is_valid = True
                else:
                    state = loc.rpartition(',')[0].strip()
                    if (state in state_name_list_lower) or (state in state_abb_list_lower):
                        if state in state_name_list_lower:
                            states.append(state)
                        else:
                            states.append(state_dict[state])
                        counties.append(None)
                        cities.append(None)
                        is_valid = True
                    
            elif loc in state_abb_list_lower:
                states.append(state_dict[loc])
                cities.append(None)
                counties.append(None)
                is_valid = True
            elif loc in state_name_list_lower:
                states.append(loc)
                cities.append(None)
                counties.append(None)
                is_valid = True
            elif loc in list(county_dict_unique.keys()):
                states.append(county_dict_unique[loc])
                counties.append(loc)
                cities.append(None)
                is_valid = True
            elif loc in list(city_dict_unique.keys()):
                states.append(city_dict_unique[loc])
                cities.append(loc)
                counties.append(None)
                is_valid = True
    if is_valid == False:
        states.append(None)
        cities.append(None)
        counties.append(None)

In [179]:
has_state = []
has_county = []
has_city = []
for s,county,city in zip(states,counties,cities):
    if s is None:
        has_state.append(False)
    else:
        has_state.append(True)
    if county is None:
        has_county.append(False)
    else:
        has_county.append(True)
    if city is None:
        has_city.append(False)
    else:
        has_city.append(True)

In [180]:
the_tweets['state'] = states
the_tweets['city'] = cities
the_tweets['county'] = counties
the_tweets['has_state'] = has_state
the_tweets['has_county'] = has_county
the_tweets['has_city'] = has_city

In [181]:
tweets_df.to_csv('tweets_df_with_location_marked.csv')

In [182]:
the_tweets.to_csv('the_tweets_with_location_marked.csv')

# 6 Match the weights

## 6.0 (Optional) Import `tweets_df` and `the_tweets`

In [3]:
tweets_df = pd.read_csv('tweets_df_with_location_marked.csv')
the_tweets = pd.read_csv('the_tweets_with_location_marked.csv')

C:\Users\86188\AppData\Local\Temp\ipykernel_73416\4241596199.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets_df = pd.read_csv('tweets_df_with_location_marked.csv')


In [8]:
weights = pd.read_csv('../lexicon weighting/final_weights.csv')

In [9]:
tweets_df = tweets_df.merge(weights, how='left', left_on='Keyword', right_on = 'Unnamed: 0')

# 7 Matches and Indexes with "the_tweets"

In [4]:
tweets_have_state = tweets_df['Keyword'][tweets_df['has_state']]
pd.Series(collections.Counter(tweets_have_state)).to_csv('distribution_has_state.csv')

In [5]:
tweets_have_county = tweets_df['Keyword'][tweets_df['has_county']]
pd.Series(collections.Counter(tweets_have_county)).to_csv('distribution_has_county.csv')

In [6]:
tweets_have_city = tweets_df['Keyword'][tweets_df['has_city']]
pd.Series(collections.Counter(tweets_have_city)).to_csv('distribution_has_city.csv')

## 7.1 State level

In [15]:
tweets_state = tweets_df[tweets_df['has_state']]

In [11]:
len(tweets_state)

309083

In [16]:
the_state = the_tweets[['state','Text']].groupby('state').count()
the_state = the_state.rename(columns={'Text':'n_the_tweets'})

In [17]:
tweets_state.columns

Index(['Unnamed: 0_x', 'Datetime', 'Tweet Id', 'Text', 'User', 'User_location',
       'Place', 'Coordinates', 'Category', 'Keyword', 'keep_list', 'state',
       'city', 'county', 'has_state', 'has_county', 'has_city', 'Unnamed: 0_y',
       'weight'],
      dtype='object')

In [18]:
the_state_count_by_keyword = tweets_state.groupby(['state','Keyword']).size()

In [23]:
the_state_count_by_keyword.to_csv('the_state_count_by_keyword.csv')

In [188]:
the_state_weight = tweets_state[['state','weight']].groupby('state').sum()
the_state_count = tweets_state[['state','weight']].groupby('state').count()
the_state_count = the_state_count.rename(columns={'weight':'count'})

In [189]:
the_state_mis = the_state_weight.merge(the_state_count, left_index=True, right_index=True)
the_state_mis = the_state_mis.merge(the_state, how='left', left_index=True, right_index=True)
the_state_mis['mis_index_weight'] = the_state_mis['weight']/the_state_mis['n_the_tweets']
the_state_mis['mis_index_count'] = the_state_mis['count']/the_state_mis['n_the_tweets']

In [191]:
the_state_mis.to_csv('the_state_mis.csv')

## 7.2 County level

In [20]:
tweets_county = tweets_df[tweets_df['has_county']]
len(tweets_county)

59121

In [21]:
the_county_count_by_keyword = tweets_state.groupby(['state','county','Keyword']).size()
the_county_count_by_keyword

state    county    Keyword  
alabama  chilton   bitch        1
         crenshaw  bitch        6
                   whore        1
         cullman   bitch        3
                   fucker       3
                               ..
wyoming  laramie   dumb fuck    1
                   pussy        2
         park      bitch        1
         sheridan  bitch        2
                   dumb fuck    1
Length: 2075, dtype: int64

In [22]:
the_county_count_by_keyword.to_csv('the_county_by_keyword.csv')

In [193]:
the_county_weight = tweets_county[['state','county','weight']].groupby(['state','county']).sum()
the_county_count = tweets_county[['state','county','weight']].groupby(['state','county']).count()
the_county_count = the_county_count.rename(columns={'weight':'count'})

In [194]:
len(county_mis)

849

In [195]:
the_county = the_tweets[['state','county','Text']].groupby('county').count()
the_county = the_county.rename(columns={'Text':'n_the_tweets'})

In [196]:
the_county_mis = the_county_weight.merge(the_county_count, left_index=True, right_index=True)
the_county_mis = the_county_mis.merge(the_county, how='left', left_index=True, right_index=True)
the_county_mis['mis_index_weight'] = the_county_mis['weight']/the_county_mis['n_the_tweets']
the_county_mis['mis_index_count'] = the_county_mis['count']/the_county_mis['n_the_tweets']

In [197]:
the_county_mis.to_csv('the_county_mis.csv')

## 7.3 City level

In [198]:
tweets_city = tweets_df[['Keyword','city','weight']][tweets_df['has_city']]
len(tweets_city)

70734

In [199]:
city_mis = tweets_city[['city','weight']].groupby('city').sum()

In [200]:
len(city_mis)

3824

In [201]:
the_city = the_tweets[['city','Text']].groupby('city').count()

In [202]:
the_city_mis = city_mis.merge(the_city, how='left',left_index=True, right_index=True)

In [203]:
collections.Counter(the_city_mis['_merge'])

Counter({'both': 2778, 'left_only': 1046})

In [412]:
the_city_mis['mis_index'] = the_city_mis['weight']/the_city_mis['Text']

In [413]:
the_city_mis.to_csv('the_city_mis.csv')

-----

# A Matches and Indexes with adm population data

## A (1) Some more processing before matching

In [270]:
tweets_df.replace('hawaiʻi','hawaii',inplace=True)

## 8.2 Weights

In [106]:
weights = pd.read_csv('../lexicon weighting/final_weights.csv')

In [108]:
tweets_df = tweets_df.merge(weights, how='left', left_on='Keyword', right_on = 'Unnamed: 0')

## 8.3 State level

In [271]:
tweets_state = tweets_df[['Keyword','state','weight']][tweets_df['has_state']]

In [272]:
len(tweets_state)

315151

In [414]:
state_mis = tweets_state[['state','weight']].groupby('state').sum()
state_mis_pop = state_mis.merge(state_population_series, how='left', left_index=True, right_index=True)
state_mis_pop['mis_index'] = state_mis_pop['weight']/state_mis_pop['population']

In [415]:
state_mis_pop.to_csv('state_mis_pop.csv')

## 5.4 County level

In [336]:
tweets_county = tweets_df[['Keyword','county','weight']][tweets_df['has_county']]
len(tweets_county)

90598

In [330]:
county_mis = tweets_county[['county','weight']].groupby('county').sum()

In [344]:
len(county_mis)

849

In [345]:
county_mis_merged = county_mis.merge(county_population, how='left',left_index=True, right_on='county',indicator=True)

In [346]:
collections.Counter(county_mis_merged['_merge'])

Counter({'both': 1796, 'left_only': 33})

In [349]:
county_mis_merged['mis_index_1'] = county_mis_merged['weight']/county_mis_merged['CENSUS2010POP']
county_mis_merged['mis_index_2'] = county_mis_merged['weight']/county_mis_merged['POPESTIMATE2019']

In [416]:
county_mis_merged.to_csv('county_mis_pop.csv')

## 5.5 City level

In [359]:
tweets_city = tweets_df[['Keyword','city','weight']][tweets_df['has_city']]
len(tweets_city)

131504

In [351]:
city_mis = tweets_city[['city','weight']].groupby('city').sum()

In [354]:
len(city_mis)

4761

In [366]:
city_mis_merged = city_mis.merge(city_population, how='outer',left_index=True, right_on='city',indicator=True)

In [367]:
collections.Counter(city_mis_merged['_merge'])

Counter({'left_only': 4254, 'both': 534, 'right_only': 254})

In [362]:
city_mis_merged['mis_index_1'] = city_mis_merged['weight']/city_mis_merged['2010 Census']
city_mis_merged['mis_index_2'] = city_mis_merged['weight']/city_mis_merged['2019 Estimate']